# Search Latitude And Longtitude For Some Website Program

### Import Library

In [1]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from chromedriver_py import binary_path
from selenium import webdriver
from bs4 import BeautifulSoup
import time

### Input Location

In [2]:
key_word = 'Siam Paragon'

### Standard Open Website Function

In [3]:
def open_web(web_name):
    options = Options()
    options.headless = True
    options.add_argument("user-data-dir=C:\\Users\\TPCS\\AppData\\Local\\Google\\Chrome\\User Data\\Default")
    service_object = Service(binary_path)
    driver = webdriver.Chrome(service=service_object,options=options)
    driver.maximize_window()
    driver.get(web_name)
    time.sleep(3)
    return driver

### Find Latitude, Longtitude and Property Function

In [4]:
def find_lat_lng_gmap(key_word):
    web_name = "Search Engine Website"
    driver = open_web(web_name)
    time.sleep(2)
    result = {
        'Error':'',
        'Key Word':key_word,
        'latitude':0,
        'longtitude':0,
        'address':'',
        "postcode":'',
        "phone":'',
        "website":'',
        'Google Map URL':''
    }
    try:
        search_areas = driver.find_element(By.XPATH, 'Search_Areas_XPath')
        search_areas.clear()
        search_areas.send_keys(key_word)
        search_areas.submit()
        time.sleep(2)

        gmap_url = driver.current_url
        lat,lng=gmap_url.split('@')[1].split(',')[:2]
        
        data = driver.page_source
        soup = BeautifulSoup(data, 'html.parser')      

        address = soup.find('button', {'data-item-id': 'address'}).get('aria-label', '').replace('ที่อยู่: ', '').strip()

        address_list = address.split(' ')
        postcode = [addr if (len(str(addr)) == 5) and (addr).isdigit() else '' for addr in address_list].pop()
        
        phone_button = soup.find('button', {'data-item-id': lambda x: x and x.startswith('phone:tel')})
        phone = phone_button.get('aria-label', '').replace('โทรศัพท์:', '').replace(' ','').strip() if phone_button else None

        website_tag = soup.find('a', {'data-item-id': 'authority'})
        website = website_tag.get('href', '') if website_tag else None
        time.sleep(1)
        
        glink_button = driver.find_element(By.XPATH, "Google_Link_XPath")
        glink_button.click()
        time.sleep(2)
        
        gmap_url = driver.find_element(By.XPATH, "Google_Map_URL_XPath").get_attribute("value")       
        time.sleep(1)

        result['latitude'] = lat
        result['longtitude'] = lng
        result['address'] = address
        result['postcode'] = postcode
        result['phone'] = phone
        result['website'] = website
        result['Google Map URL'] = gmap_url
    except Exception as e:
        result['Error'] = e
    finally:
        driver.quit()
        return result

In [5]:
result = find_lat_lng_gmap(key_word)

C:\Users\intrust\AppData\Local\Temp\ipykernel_10972\2989429425.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [6]:
result

{'Error': '',
 'Key Word': 'Siam Paragon',
 'latitude': '13.7457749',
 'longtitude': '100.5344017',
 'address': '991/1 ถ. พระรามที่ 1 แขวงปทุมวัน เขตปทุมวัน กรุงเทพมหานคร 10330',
 'postcode': '10330',
 'phone': '026901000',
 'website': 'https://www.siamparagon.co.th/',
 'Google Map URL': 'https://maps.app.goo.gl/w42Kc7D3VQgYEbsi6'}